## 概要
Greengrass を用いた機械学習アプリの開発と配布を行うハンズオンです。

* EC2 をエッジデバイスと見立てて、エッジ側で ML 推論するアプリを開発・配布します。
* 実際の開発を模して、開発機(1台)と、ステージング機(1台)、本番機(2台)を作成します。
* 開発機でアプリ開発、ステージング機で配布と動作のチェック、本番機でアプリを一括配布することを体験します。
* 使用する ML モデルは [build_mnist_classifier.ipynb](./build_mnist_classifier.ipynb) で作成したものを利用します(すでに配置済)
* このノートブックを実行するコンピューティングリソースには、下記ポリシーがアタッチされている前提です。
    * `AmazonEC2FullAccess`
    * `AmazonSSMFullAccess`
    * `AmazonS3FullAccess`
    * `IAMFullAccess`
    * `AWSGreengrassFullAccess`
    * `AWSIoTFullAccess`
    * `AmazonEC2ContainerRegistryFullAccess`

## 構成
* ハンズオンの流れ 
  1. エッジデバイスを模した EC2 のセットアップと Greengrass CLI をインストール
  2. 開発機 でローカル開発＆ローカルデプロイ
  3. クラウド側でコンポーネント作成
  4. ステージング機 にデプロイ & 確認
  5. 本番機 2台にデプロイ
* 開発するもの
  1. Publisherで撮像して画像ファイルのパスをIPCでパブリッシュ(実態はGANで画像生成）
  2. Subscriber で画像に写った数字が奇数か偶数か判定
  3. 1と2の機能をまとめて MQTT でパブリッシュするコンテナアプリ
  4. 3 のモデルのアップデートしたコンテナアプリ
  
ハンズオンが完了するとこのような構成が出来上がります
![ハンズオンの最終形](./image/image01.png)

## 手順
1. [エッジデバイスの事前セットアップ](#エッジデバイスの事前セットアップ)
    1. [作成する Greengrass デバイスの定義](#作成する-Greengrass-デバイスの定義)
    2. [EC2 で利用するキーペアを作成](#EC2-で利用するキーペアを作成)
    3. [EC2 のロールを作成](#EC2-のロールを作成)
    4. [EC2 起動](#EC2-起動)
    5. [Greengrass Artifact 用 S3 バケットを作成](#Greengrass-Artifact-用-S3-バケットを作成)
    6. [Greengrass core が作成・使用するロールとポリシーが存在する場合の事前削除](#Greengrass-core-が作成・使用するロールとポリシーが存在する場合の事前削除)
    7. [Greengrass core が使用するロール作成](#Greengrass-core-が使用するロール作成)
    8. [RoleAlias の作成](#RoleAlias-の作成)
2. [Greengrass CLI をエッジ (EC2) にインストール](#Greengrass-CLI-をエッジ-(EC2)-にインストール)
    1. [既存 thing と group を削除](#既存-thing-と-group-を削除)
    2. [EC2 に Greengrass CLI をインストールするコマンドの準備](#EC2-に-Greengrass-CLI-をインストールするコマンドの準備)
    3. [インストールコマンドを実行](#インストールコマンドを実行)
3. [Publisher / Subscriber の開発](#Publisher-/-Subscriber-の開発)
    1. [Publisher のローカルデプロイ](#Publisher-のローカルデプロイ)
        1. [開発機で Publisher のローカルコンポーネント作成とローカルデプロイ](#開発機で-Publisher-のローカルコンポーネント作成とローカルデプロイ)
        2. [Publisher のデプロイ結果を確認](#Publisher-のデプロイ結果を確認)
        3. [デプロイした Publisher コンポーネントの動作を確認](#デプロイした-Publisher-コンポーネントの動作を確認)
    2. [Subscriber のローカルデプロイ](#Subscriber-のローカルデプロイ)
        1. [開発機で Subscriber のローカルコンポーネント作成とローカルデプロイ](#開発機で-Subscriber-のローカルコンポーネント作成とローカルデプロイ)
        2. [Subscriber のデプロイ結果を確認](#Subscriber-のデプロイ結果を確認)
        3. [デプロイした Subscriber コンポーネントの動作を確認](#デプロイした-Subscriber-コンポーネントの動作を確認)
    3. [Greengrass クラウドからデプロイ](#Greengrass-クラウドからデプロイ)
        1. [既存コンポーネントが存在する場合の削除](#既存コンポーネントが存在する場合の削除)
        2. [Publisher のコンポーネントを作成](#Publisher-のコンポーネントを作成)
        3. [Subscriber のコンポーネントを作成](#Subscriber-のコンポーネントを作成)
        4. [Publisher / Subscriber のクラウドデプロイ](#Publisher-/-Subscriber-のクラウドデプロイ)
            1. [ステージング機へのデプロイ](#ステージング機へのデプロイ)
            2. [ステージング機へのデプロイ結果確認](#ステージング機へのデプロイ結果確認)
            3. [本番機へのデプロイ](#本番機へのデプロイ)
            4. [本番機へのデプロイ結果確認](#本番機へのデプロイ結果確認)
4. [コンテナを利用したコンポーネントの開発](#コンテナを利用したコンポーネントの開発)
    1. [コンテナイメージのビルド](#コンテナイメージのビルド)
    2. [ECR へコンテナイメージをプッシュ](#ECR-へコンテナイメージをプッシュ)
    3. [コンテナを使ったコンポーネントを開発機にローカルデプロイ](#コンテナを使ったコンポーネントを開発機にローカルデプロイ)
        1. [コンテナローカルデプロイ用 Recipe を作成](#コンテナローカルデプロイ用-Recipe-を作成)
        2. [開発機でコンテナコンポーネントのローカルデプロイ](#開発機でコンテナコンポーネントのローカルデプロイ)
        3. [ローカルデプロイした IoTPublisher が MQTTで パブリッシュした内容を確認する](#ローカルデプロイした-IoTPublisher-が-MQTT-でパブリッシュした内容を確認する)
    4. [コンテナを使ったコンポーネントをステージング機にクラウドデプロイ](#コンテナを使ったコンポーネントをステージング機にクラウドデプロイ)
        1. [既存コンテナコンポーネントが存在する場合の削除](#既存コンテナコンポーネントが存在する場合の削除)
        2. [コンテナを用いたコンポーネントの作成](#コンテナを用いたコンポーネントの作成)
        3. [ステージング機のデプロイの改定](#ステージング機のデプロイの改定)
        4. [ステージング機にクラウドデプロイした IoTPublisher が MQTT でパブリッシュした内容を確認する](#ステージング機にクラウドデプロイした-IoTPublisher-が-MQTT-でパブリッシュした内容を確認する)
5. [コンテナを利用したコンポーネントの更新](#コンテナを利用したコンポーネントの更新)
    1. [コンテナイメージの更新](#コンテナイメージの更新)
    2. [ECR へ更新したコンテナイメージをプッシュ](#ECR-へ更新したコンテナイメージをプッシュ)
    3. [更新したコンテナを使ったコンポーネントを開発機にローカルデプロイ](#更新したコンテナを使ったコンポーネントを開発機にローカルデプロイ)
        1. [更新したコンテナのローカルデプロイ用 Recipe を作成](#更新したコンテナのローカルデプロイ用-Recipe-を作成)
        2. [開発機で更新したコンテナのローカルデプロイ](#開発機で更新したコンテナのローカルデプロイ)
        3. [更新したコンテナのローカルデプロイ結果を確認（開発機）](#更新したコンテナのローカルデプロイ結果を確認（開発機）)
    4. [更新したコンテナを使ったコンポーネントをクラウドデプロイ](#更新したコンテナを使ったコンポーネントをステージング機にクラウドデプロイ)
        1. [更新したコンテナを用いたコンポーネントの作成](#更新したコンテナを用いたコンポーネントの作成)
        2. [更新したコンテナを用いたステージング機のデプロイの改定](#更新したコンテナを用いたステージング機のデプロイの改定)
        3. [ステージング機で更新したコンテナの MQTT 動作確認](#ステージング機で更新したコンテナの-MQTT-動作確認)

In [ ]:
!date

In [ ]:
import boto3, json, yaml, os
from time import sleep

In [ ]:
# region 取得
region = boto3.session.Session().region_name

# 使用するサービスのクライアント生成
ec2_client = boto3.client('ec2', region_name=region)
ssm_client = boto3.client('ssm', region_name=region)
s3_client = boto3.client('s3', region_name=region)
iam_client = boto3.client('iam')
ggv2_client = boto3.client('greengrassv2', region_name=region)
iot_client = boto3.client('iot', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)

## エッジデバイスの事前セットアップ
EC2 をエッジデバイスとして扱うためのセットアップ

### 作成する Greengrass デバイスの定義 
* 本ハンズオンではEC2 4 台をエッジデバイスとして扱うため、AWS IoT Core 上のグループ、名前を設定する
* クラウドでの名前の登録自体は Greengrass CLI をインストールするときに行われるが、ここでは変数にそれぞれの名前を格納する

In [ ]:
# 開発環境
develop_group = {
    "name": "gg-ml-iot-thing-development-group",
    "thing_names":['gg-ml-iot-thing-development']
}
# ステージング環境
staging_group = {
    "name": "gg-ml-iot-thing-staging-group",
    "thing_names":['gg-ml-iot-thing-staging']
}
# 本番環境
production_group = {
    "name": "gg-ml-iot-thing-production-group",
    "thing_names":[
        'gg-ml-iot-thing-production1',
        'gg-ml-iot-thing-production2'
    ]
}
all_groups = [develop_group, staging_group, production_group]
device_name_list=[y for x in all_groups for y in x["thing_names"]]
print(device_name_list)
print(f"作成するデバイスの数: {len(device_name_list)}")

### EC2 で利用するキーペアを作成
* 本ハンズオンの本筋とは関係なく、EC2特有の操作
* sshでログインできるようにするための「key-pair-for-greengrass-on-ec2」というsshキーを作成する
* 同名のキーがあった場合は削除して作り直す

In [ ]:
key_pair_name = 'key-pair-for-greengrass-on-ec2'
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x : x['KeyName'], key_pairs['KeyPairs']))

if key_pair_name in key_names:
    ec2_client.delete_key_pair(KeyName=key_pair_name)

ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_pair_name,
)

key_pair = str(ec2_key_pair['KeyMaterial'])
with open('ec2-key-pair.pem','w') as f:
    f.write(key_pair)

### Greengrass Artifact 用 S3 バケットを作成
* Greengrass の Artifact(Greengrassで動かすアプリのファイル群) を保存する S3 Bucket を作成
* 他にも EC2 へ SSM 経由で流し込んだ出力結果を保存するのにも使用
* Greengrass が使用するロールにこのバケットへのアクセス権限を与える必要があるため、この段階でバケット名を決めて作成する

![バケット作成](./image/image02.png)

In [ ]:
# バケットが存在している場合は事前に削除
bucket = 'type great bucket name' # <- グローバルでユニークなS3バケット名を入力
!aws s3 rb s3://{bucket} --force
# バケット作成
location = {'LocationConstraint': region}
if region == 'us-east-1':
    response = s3_client.create_bucket(
        Bucket = bucket
    )
else:
    response = s3_client.create_bucket(
        Bucket = bucket,
        CreateBucketConfiguration = location
    )

In [ ]:
!aws s3 ls | grep {bucket}

### EC2 のロール及びインラインポリシーを作成
* 本ハンズオンの本筋とは関係なく、EC2特有の操作
* EC2にシェルコマンドを送り込むのにsession_managerというサービスを使うため、それらの権限が付与されたロールを作成
* Greengrass CLI をインストールする際に必要なポリシーもあるので(IAMFullAccess,AWSIoTFullAccess)、別途そのポリシーを有したIAMユーザやロールを発行して、ACCESS_KEYを設定するか、[IoTクライアント証明書を利用](https://docs.aws.amazon.com/ja_jp/iot/latest/developerguide/device-certs-create.html)するなどをしておく

In [ ]:
# 定数定義
ec2_role_name = 'EC2GreengrassRole'
ec2_inline_policy_name = 'GreengrassEc2InlinePolicy'
ec2_profile=ec2_role_name + "Profile"

ec2_role_attach_policy_arn_list = [
    "arn:aws:iam::aws:policy/AmazonEC2FullAccess",
    "arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore",
    "arn:aws:iam::aws:policy/IAMFullAccess",
    "arn:aws:iam::aws:policy/AWSIoTFullAccess",
    "arn:aws:iam::aws:policy/AWSGreengrassFullAccess",
    'arn:aws:iam::aws:policy/service-role/AmazonEC2RoleforSSM',
    'arn:aws:iam::aws:policy/AmazonSSMDirectoryServiceAccess',
    'arn:aws:iam::aws:policy/CloudWatchAgentServerPolicy',
]

In [ ]:
# 既存ポリシーのデタッチ、ポリシーバージョンの削除、ポリシーの削除
# 存在しない場合はexceptでスキップされる

# インラインポリシーの削除
policies = iam_client.list_policies()
for policy in policies['Policies']:
    policy_arn = policy['Arn'] if policy['PolicyName'] in ec2_inline_policy_name else None
    if policy_arn:
        # Policy をデタッチ
        try:
            response = iam_client.detach_role_policy(
                RoleName=ec2_role_name,
                PolicyArn=policy_arn
            )
            print(f'detach {ec2_role_name} {policy_arn}')
            print(respoonse)
        except:
            print('did not detach role policy')
        # Policy バージョンの削除
        try:
            for policy_version in iam_client.list_policy_versions(PolicyArn=policy_arn)['Versions']:
                response = iam.delete_policy_version(PolicyArn=policy_arn,VersionId=policy_version['VersionId'])
                print(f'delete policy version {policy_arn} {policy_version["VersionId"]}')
                print(response)
        except:
            print('did not delete policy version')
        # Policy を削除
        try:
            response = iam_client.delete_policy(PolicyArn=policy_arn)
            print(f'delete policy {policy_arn}')
            print(response)
        except:
            print('did not delete policy')

# その他ポリシーの削除
for policy_arn in ec2_role_attach_policy_arn_list:                      
    try:
        response = iam_client.detach_role_policy(RoleName=ec2_role_name,PolicyArn=policy_arn)
        print(f'detach ec2 role {policy_arn}')
        print(json.dumps(response, indent=2))
    except:
        print(f'could not detach {policy_arn}')
                      


# インスタンスプロファイルを削除
try:
    response = iam_client.remove_role_from_instance_profile(
        InstanceProfileName=ec2_profile,
        RoleName=ec2_role_name
    )
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to remove role from instance profile')

# ロールの削除
try:
    response = iam_client.delete_role(RoleName = ec2_role_name)
    print('delete ec2 role')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete role')
try:
    response = iam_client.delete_instance_profile(InstanceProfileName=ec2_profile)
    print('delete instance profile')
    print(json.dumps(response, indent=2))
except:
    print('nothing to delete instance profile')

In [ ]:
sleep(30)
# インラインポリシーの作成
s3_get_from_ec2_policy_doc = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Action': [
                's3:GetObject',
                's3:PutObject',
                's3:PutObjectAcl',
            ],
            'Resource': [
                f'arn:aws:s3:::{bucket}/*',
                f'arn:aws:s3:::aws-ssm-{region}/*',
                f'arn:aws:s3:::aws-windows-downloads-{region}/*',
                f'arn:aws:s3:::amazon-ssm-{region}/*',
                f'arn:aws:s3:::amazon-ssm-packages-{region}/*',
                f'arn:aws:s3:::{region}-birdwatcher-prod/*',
                f'arn:aws:s3:::aws-ssm-distributor-file-{region}/*',
                f'arn:aws:s3:::aws-ssm-document-attachments-{region}/*',
                f'arn:aws:s3:::patch-baseline-snapshot-{region}/*'
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "ssmmessages:CreateControlChannel",
                "ssmmessages:CreateDataChannel",
                "ssmmessages:OpenControlChannel",
                "ssmmessages:OpenDataChannel"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetEncryptionConfiguration"
            ],
            "Resource": "*"
        }
    ]
}
response = iam_client.create_policy(
    PolicyName=ec2_inline_policy_name,
    PolicyDocument=json.dumps(s3_get_from_ec2_policy_doc),
    Description='s3 get from ec2 policy',
)
ec2_inline_policy_arn = response['Policy']['Arn']
ec2_inline_policy_name = response['Policy']['PolicyName']
print(f'作成した policy の\narn は "{ec2_inline_policy_arn}"')
print(f'名前は "{ec2_inline_policy_name}"\nです')

In [ ]:
# ロールの作成
assume_role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service": "ec2.amazonaws.com"},"Action": "sts:AssumeRole"}]
}
response = iam_client.create_role(
    RoleName = ec2_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
    Description='edge equivalent ec2 role',
    MaxSessionDuration=3600*12 # 12時間
)
print(response)

In [ ]:
# ポリシーをロールにアタッチ
# inline policy
response = iam_client.attach_role_policy(
    RoleName=ec2_role_name,
    PolicyArn=ec2_inline_policy_arn
)
print(json.dumps(response, indent=2))

# managed policy
for policy_arn in ec2_role_attach_policy_arn_list:
    response = iam_client.attach_role_policy(
        RoleName=ec2_role_name,
        PolicyArn=policy_arn
    )
    print(f'attach policy_arn')
    print(json.dumps(response, indent=2))

In [ ]:
response = iam_client.create_instance_profile(InstanceProfileName=ec2_profile)
response = iam_client.add_role_to_instance_profile(
    RoleName=ec2_role_name,
    InstanceProfileName=ec2_profile
)

### EC2 起動
* 本ハンズオンの本筋とは関係ない
* user_dataで流しんでいるコマンドは下記を実現するためのものなので、エッジデバイス側で予め設定すること
  * python3 コマンドで起動する Python のバージョンが 3.8 にする
  * awscli, unzip, zip, docker, pip3 のインストール

![EC2の起動](./image/image03.png)

In [ ]:
work_dir = '/root/work'
exec_log_file = os.path.join(work_dir,'1_ec2_userdata.log')
detail_log_file = os.path.join(work_dir,'2_ec2_detail_userdata.log')
user_data = f"""#!/bin/bash
export work_dir={work_dir}
export exec_log_file={exec_log_file}
export detail_log_file={detail_log_file}
whoami >> $exec_log_file
mkdir -p $work_dir
cd $work_dir
pwd >> $exec_log_file

date >> $exec_log_file
echo "apt update" >> $exec_log_file
apt update -y >> $detail_log_file

date >> $exec_log_file
echo "apt install" >> $exec_log_file
apt install python3-pip unzip awscli zip docker docker.io -y >> $detail_log_file

date >> $exec_log_file
echo "pip3 install" >> $exec_log_file
pip3 install awscli >> $detail_log_file

date >> $exec_log_file
echo "apt install python3.8" >> $exec_log_file
apt install -y python3.8 >> $detail_log_file

# Python3.8 をデフォルトに
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 2
echo 2 | update-alternatives --config python3
python3 --version >> $exec_log_file
echo completed  >> $exec_log_file
date >> $exec_log_file

"""
print(user_data)

In [ ]:
# ubuntu の AMI 集
ami_map = {
    'af-south-1':'ami-075520a13dc442833',
    'ap-east-1':'ami-0a4e8c6468d92c210',
    'ap-northeast-1':'ami-0620ef6ee255ea559',
    'ap-south-1':'ami-0c0490c60db84298f',
    'ap-southeast-1':'ami-09b5aaebc21c273eb',
    'ca-central-1':'ami-05c0545bdc0bbff91',
    'eu-central-1':'ami-08db015a4afd75546',
    'eu-north-1':'ami-05916395833cb9690',
    'eu-south-1':'ami-03839a9ab91b19abc',
    'eu-west-1':'ami-0f5b07b31937d4275',
    'me-south-1':'ami-019ba1b47cfb5211e',
    'sa-east-1':'ami-092dc94ff1b65b5c2',
    'us-east-1':'ami-0e4d932065378fd3d',
    'us-west-1':'ami-05620e35978c63272',
    'ap-northeast-2':'ami-0f77aba17625db03b',
    'ap-southeast-2':'ami-0b643a2ce5f48199a',
    'eu-west-2':'ami-0a14509f661bf2964',
    'us-east-2':'ami-063e88ad6c9af427d',
    'us-west-2':'ami-0b7d93899b51ff83b',
    'ap-northeast-3':'ami-06947c2f8a47debf3',
    'eu-west-3':'ami-0b722faecb702e094',
}

In [ ]:
sleep(30) # instance profile が反映されるまで待つ
ec2_instance = ec2_client.run_instances(
    BlockDeviceMappings=[{'Ebs':{'VolumeSize':20},"DeviceName" : '/dev/xvda'}],
    ImageId=ami_map[region],
    MinCount=len(device_name_list),# [development/staging/production*2]
    MaxCount=len(device_name_list),
    InstanceType='c5.large',
    KeyName=key_pair_name,
    IamInstanceProfile={'Name': ec2_profile},
    UserData = user_data
)
instance_id_list = [instance['InstanceId'] for instance in ec2_instance['Instances']]
except_counter = 0
while True:
    try:
        status_list = ec2_client.describe_instance_status(InstanceIds=instance_id_list)['InstanceStatuses']
        print([status['InstanceState']['Name'] for status in status_list])
        if [status['InstanceState']['Code'] for status in status_list] == [16]*4:
            break
        else:
            sleep(5)
    except:
        print('except')
        except_counter+=1
        if except_counter > 60:
            print('maximum except')
            break
        else:
            sleep(1)

In [ ]:
# EC2の名前のタグを設定して、EC2のコンソールから見やすくする
for instance_id,tag_value in zip(instance_id_list,device_name_list):
    ec2_client.create_tags(
        Resources = [instance_id],
        Tags = [{'Key':'Name','Value':tag_value}]
    )

### Greengrass core が作成・使用するロールとポリシーが存在する場合の事前削除
* 本ハンズオンの本筋とは関係ない
* 実行前に環境をクリーニング
* 既存の場合を想定してロールとポリシーは削除して、そのあと作り直す

In [ ]:
greengrass_role_name = 'MLIoTGGV2TESRole'
greengrass_policy_names = [
    'MLIoTGGV2TESArtifactPolicy',   # ユーザ作成
    'MLIoTGGV2TESRoleAccess'        # システム作成
]
greengrass_role_alias = greengrass_role_name + "Alias"

In [ ]:
# Role Aliasの削除
try:
    response = iot_client.delete_role_alias(
        roleAlias=greengrass_role_alias
    )
except:
    pass

In [ ]:
try:
    response = iam_client.list_attached_role_policies(
        RoleName=greengrass_role_name
    )
    print(response["AttachedPolicies"])
except:
    print('No attachedPolicies')
    response = None

    
if response: 
    for policy in response["AttachedPolicies"]:
        policy_arn = policy["PolicyArn"]
        try:
            response = iam_client.detach_role_policy(
                RoleName=greengrass_role_name,
                PolicyArn=policy_arn
            )
            print(f'detach {greengrass_role_name} {policy_arn}')
            print(respoonse)
        except:
            print('did not detach role policy')

        try:
            for policy_version in iam_client.list_policy_versions(PolicyArn=policy_arn)['Versions']:
                response = iam_client.delete_policy_version(PolicyArn=policy_arn,VersionId=policy_version['VersionId'])
                print(f'delete policy version {policy_arn} {policy_version["VersionId"]}')
                print(response)
        except:
            print('did not delete policy version')

        try:
            response = iam_client.delete_policy(PolicyArn=policy_arn)
            print(f'delete policy {policy_arn}')
            print(response)
        except:
            print('did not delete policy')
else:
    pass

# ロールを削除
try:
    response = iam_client.delete_role(RoleName = greengrass_role_name)
    print(f'delete role {greengrass_role_name}')
    print(response)
except:
    print('did not delete role')

### Greengrass CLI が使用するロール作成
* Greengrass CLI が Greengrass のサービスや、ECR や S3 を利用できるようにするためのロールの作成とポリシーのアタッチ
* 使用している API は以下の通り(すべて IAM)
  * [create_role](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.create_role)
  * [create_policy](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.ServiceResource.create_policy)
  * [attach_role_policy](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#IAM.Client.attach_role_policy)


In [ ]:
assume_role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service": "credentials.iot.amazonaws.com"},"Action": "sts:AssumeRole"}]
}
response = iam_client.create_role(
    RoleName = greengrass_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document),
    Description='using Greengrass ml inference',
    MaxSessionDuration=3600*12 # 12時間
)
role_arn = response['Role']['Arn']
role_name = response['Role']['RoleName']

print(f'作成した role の\narn は "{role_arn}"')
print(f'名前は "{role_name}"\nです')

In [ ]:
gg_role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [
      {"Effect": "Allow","Action": ["s3:*"],"Resource": f"arn:aws:s3:::{bucket}/*"},
      {"Effect": "Allow","Action": ["ecr:GetAuthorizationToken","ecr:BatchGetImage","ecr:GetDownloadUrlForLayer","iot:Connect","iot:Publish"],"Resource": ["*"]}

  ]
}

response = iam_client.create_policy(
    PolicyName=greengrass_policy_names[0],
    PolicyDocument=json.dumps(gg_role_policy_document),
    Description='for GreengrassV2Role policy',
)
policy_arn = response['Policy']['Arn']
policy_name = response['Policy']['PolicyName']
print(f'作成した policy の\narn は "{policy_arn}"')
print(f'名前は "{policy_name}"\nです')

In [ ]:
# Policy attach
response = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)
print(json.dumps(response, indent=2))

### RoleAlias の作成
* Greengrass のデバイスは AWS IoT のX.509証明書を利用したロールエイリアスで、先程作成したロールを代替させることができる  
  = ロールエイリアスで、AWS のサービス外でもロールと同じ機能を実現できるようになる
* ロールエイリアスは事前に作成しておく
* 使用している API は [create_role_alias](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iot.html#IoT.Client.create_role_alias) (IoT)

In [ ]:
response = iot_client.create_role_alias(
    roleAlias=greengrass_role_alias,
    roleArn=role_arn
)
print(response)

## Greengrass CLI をエッジ (EC2) にインストール
合計 4 台にインストール

* 開発機
* ステージング機
* 本番機 2 台  
![Greengrass CLI をインストール](./image/image04.png)

### 既存 thing と group を削除
* ハンズオンの本筋とは関係ない
* Greengrass CLI をインストールする際に thing と group が作成されるが、同名の thing と group は作成できないので事前に削除する

In [ ]:
# thing と type と certification を削除するヘルパー関数
def del_thing_and_group_and_cert(thing_name,thing_group_name):
    try:
        print('try deleting gg core')
        response = ggv2_client.delete_core_device(
            coreDeviceThingName=thing_name
        )
    except:
        print(f"Core don't exists: {thing_name}")

    try:
        print('try deleting certificate')
        for principal in iot_client.list_thing_principals(thingName=thing_name)['principals']:
            response = iot_client.detach_thing_principal(thingName=thing_name,principal=principal)
            print(response)
            response = iot_client.update_certificate(certificateId=principal.split('/')[-1],newStatus='INACTIVE')
            print(response)
            response = iot_client.delete_certificate(certificateId=principal.split('/')[-1],forceDelete=True)
            print(response)
    except:
        pass

    try:
        print('try deleting thing')
        response = iot_client.delete_thing(thingName=thing_name)
        print(response)
    except:
        pass

    try:
        print('try deleting thing group')
        response = iot_client.delete_thing_group(thingGroupName=thing_group_name)
        print(response)
    except:
        pass

    # 作成しようとしている名前のthingがあったら削除
    try:
        iot_client.delete_thing(thingName=thing_name)
    except:
        pass
    try:
        iot_client.delete_thing_group(thingName=thing_group_name)
    except:
        pass

In [ ]:
# Thing groupとthingnameを組み合わせる
for thing_group in all_groups:
    for thing_name in thing_group["thing_names"]:
        print("Group: {} Thing: {}".format(thing_group["name"], thing_name))
        del_thing_and_group_and_cert(thing_name,thing_group["name"])

### EC2 に Greengrass CLI をインストールするコマンドの準備
* java を使ってインストールする
* あらかじめ設定してある下記を引数にインストールを行う
  * thing name
  * thing group
  * role
  * role alias
* dev toolsは本番機には入れない
  * ローカルデプロイすることはないため
  * staging はデバッグ用に念の為入れておく
* Greengrass CLI のインストール時のコマンド等詳細は以下を参照
  * [Greengrass CLI のインストール](https://docs.aws.amazon.com/greengrass/v2/developerguide/getting-started.html)

In [ ]:
gg_install_dir = '/greengrass/v2'

command_str_list = []
for thing_group in all_groups:
    for thing_name in thing_group["thing_names"]:
        gg_cli_tool = "--deploy-dev-tools true" if "production" not in thing_name.lower() else ""
        command_str=f"""#!/bin/bash
# su ubuntu
sudo su -
export work_dir={work_dir}
mkdir -p {work_dir}
cd {work_dir}
pwd

# userdata の完了向けに 600 秒待機
sleep 300

# ggv2 で使用する jdk をインストール
sudo apt install openjdk-11-jdk -y

# ggv2 のインストーラを入手
wget https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip
unzip greengrass-nucleus-latest.zip -d GreengrassCore
# バージョン指定する場合はこちら
# wget https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-2.1.0.zip
# unzip greengrass-2.1.0.zip -d GreengrassCore

# ggv2 をインストール
sudo -E java -Droot={gg_install_dir} -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar --aws-region {region} --thing-name {thing_name} --thing-group-name {thing_group["name"]} --component-default-user ggc_user:ggc_group --provision true --setup-system-service true  --tes-role-name {greengrass_role_name} --tes-role-alias-name {greengrass_role_alias} {gg_cli_tool}

# ggv2 のインストールはすぐに反映されないので、sleepで待つ
sleep 180            

# docker を ggc_user から実行するためにgroupに加えておく
sudo usermod -aG docker ggc_user

# ggv2 cli のバージョン確認を持ってインストールを確認する
{gg_install_dir}/bin/greengrass-cli -V
"""
        command_str_list.append(command_str)

In [ ]:
# 開発機に流すコマンド
print(command_str_list[0])

In [ ]:
# ステージング機に流すコマンド
print(command_str_list[1])

In [ ]:
# 本番機 1 に流すコマンド
print(command_str_list[2])

In [ ]:
# 本番機 2 に流すコマンド
print(command_str_list[3])

### インストールコマンドを実行
* 作成したコマンドをそれぞれのエッジデバイスで実行する
* 今回は EC2 を利用しているので、ssm の send_command API を利用して実行する
* 実際はエッジデバイスのコンソールに入って同じコマンドを実行していく

In [ ]:
# シェルコマンドをEC2に送りこみ、結果を確認するヘルパー
def send_command(bucket,instance_id,command_list):
    print(f"send_command {instance_id}")
    document_name = "AWS-RunShellScript"
    response = ssm_client.send_command(
        InstanceIds=[instance_id],
        DocumentName=document_name,
        OutputS3BucketName=bucket,
        Parameters={
            'commands': command_list
        }
    )
    print(response)
    return response['Command']['CommandId']

def check_command_result(command_id,instance_id,sleep_time=30,retry_count=60):
    print(f"check_command_result {instance_id}")
    counter = 0
    while counter < retry_count:
        try:
            response = ssm_client.get_command_invocation(
                CommandId=command_id,
                InstanceId=instance_id,
            )
            print(response['Status'])
            if response['Status'] in ['Success','Failed']:
                print(response['StandardOutputContent'])
                break
            else:
                counter += 1
                sleep(sleep_time)
        except:
            print('invocation does not exist yet')
            counter += 1
            sleep(sleep_time)
    
    prefix=f'{command_id}/{instance_id}'
    counter = 0
    retry_count=3
    while counter < retry_count:
        response = s3_client.list_objects(
            Bucket=bucket,
            Prefix=prefix
        )
        if "Contents" in response:
            break
        counter += 1

    # 標準エラー出力を確認
    key=f'{prefix}/awsrunShellScript/0.awsrunShellScript/stderr'
    print(key)
    std = {
        'stderr':'',
        'stdout':''
    }
    try:
        txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
        std['stderr'] = txt
    except:
        print('Not found stderr')

    # 標準出力を確認
    key=f'{prefix}/awsrunShellScript/0.awsrunShellScript/stdout'
    print(key)
    try:
        txt = s3_client.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
        std['stdout'] = txt
    except:
        print('Not found stdout')
    return std

In [ ]:
%%time

sleep(120) # session manager が有効化されるまで待つ
command_ids = []
for command_str,instance_id in zip(command_str_list,instance_id_list):
    print(instance_id)
    command_id = send_command(bucket, instance_id, command_str.split('\n'))
    command_ids.append(command_id)
for command_id,instance_id in zip(command_ids,instance_id_list):
    response = check_command_result(command_id,instance_id)
    print(response)

## Publisher / Subscriber の開発
### Publisher のローカルデプロイ
![Publisher のローカルデプロイ](./image/image05.png)
#### 開発機で Publisher のローカルコンポーネント作成とローカルデプロイ
* Publisher で画像を撮像（する想定で png ファイルを text.npy から生成)し、その画像のファイルパスを IPC で Publish する
* 手動で開発した体で、development に開発済みのコードを予め S3 にアップロードしておき、エッジ側でダウンロードする
* コマンド自体は session manager を使ってコマンドを遠隔で送り込む（実際はエッジデバイスのコンソールに入ってコマンドを実行する）
* ローカルでデプロイする場合は `greengrass-cli deployment create` コマンドを使う。詳細は [deployment](https://docs.aws.amazon.com/greengrass/v2/developerguide/gg-cli-deployment.html) を参照

In [ ]:
base_dir = '/ggv2'
artifact_base_dir = f'{base_dir}/components/artifacts'
artifact_dir = f'{artifact_base_dir}/com.example.Publisher/1.0.0'
recipe_dir = f'{base_dir}/components/recipes'
component_name = 'com.example.Publisher=1.0.0'

In [ ]:
# mnist のデータをダウンロード
from tensorflow.keras.datasets import mnist
import numpy as np
(_, _), (test_X, _) = mnist.load_data()
test_X = (test_X-127.5)/127.5
test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],test_X.shape[2],1))
np.save('./src/ggv2/components/artifacts/com.example.Publisher/1.0.0/test_X.npy',test_X)

In [ ]:
# artifact を S3 にアップロードする
publisher_artifact_s3_uri = f's3://{bucket}/components/artifacts/com.example.Publisher/1.0.0/'
!aws s3 cp ./src/ggv2/components/artifacts/com.example.Publisher/1.0.0 {publisher_artifact_s3_uri} --recursive
    
# recipe を S3 にアップロードする
recipe_s3_uri = f's3://{bucket}/components/recipes/'
!aws s3 cp ./src/ggv2/components/recipes/com.example.Publisher-1.0.0.yaml {recipe_s3_uri}

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
sudo mkdir -p {base_dir}
mkdir -p {artifact_dir}
cd {artifact_dir}
aws s3 cp {publisher_artifact_s3_uri} . --recursive
mkdir {recipe_dir}
cd {recipe_dir}
aws s3 cp {recipe_s3_uri}com.example.Publisher-1.0.0.yaml .
sudo chown -R ubuntu:ubuntu {base_dir}
sudo /greengrass/v2/bin/greengrass-cli deployment create --recipeDir {recipe_dir} --artifactDir {artifact_base_dir} --merge "{component_name}"
"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

### Publisher のデプロイ結果を確認
* `greengrass-cli deployment status` コマンドで確認可能
* deployment の ID が必要なため、deployment 時の出力から抽出
* あくまで Greengrass としてデプロイがうまくいったかであり、コンポーネントが正常に動作しているかは関与しない(動き始めて何秒か経過した後エラーで落ちたりしても deployment 上は SUCCEEDED と表示される)

In [ ]:
# deployment id を標準出力から取得する
deployment_id = response['stdout'].split(' ')[-1]
print(f'Deployment Id: {deployment_id}')

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
sudo /greengrass/v2/bin/greengrass-cli deployment status -i {deployment_id}
"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
sleep(60) # デプロイ完了を待つ
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

In [ ]:
print(response['stdout'])

### デプロイした Publisher コンポーネントの動作を確認
* 1 分おきに画像を`/tmp`に png 形式で出力されるはずなので、出力された画像を、エッジ → S3 → SageMaker Notebook に転送して確認する

In [ ]:
prefix='/publisher/output/development/'

command_str=f"""#!/bin/bash
latest_file=`ls -t /tmp/*png | head -n1`
aws s3 cp $latest_file s3://{bucket}{prefix}
echo $latest_file"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
sleep(60) # 60秒待てば画像はできる
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

In [ ]:
# 画像の確認、何かしらの画像が表示される

output_file_name = response['stdout'].split('\n')[-2][5:]
!aws s3 cp s3://{bucket}{prefix}{output_file_name} .
from PIL import Image
Image.open(f'./{output_file_name}')

### Subscriber のローカルデプロイ
![Subscriber のローカルデプロイ](./image/image06.png)

* 先程作成した Publisher は IPC で出力した画像のパスをパブリッシュしている
* Subscriber で Publisher のメッセージをサブスクライブして、MLを用いた画像分類器にかける
  
#### 開発機で Subscriber のローカルコンポーネント作成とローカルデプロイ
* ローカルデプロイの方法は Publisher と全く同じ
* recipe ファイルとコード一式（Artifact) をエッジ側に持っていき、デプロイする

In [ ]:
artifact_dir = f'{artifact_base_dir}/com.example.Subscriber/1.0.0'
recipe_dir = f'{base_dir}/components/recipes'
component_name = 'com.example.Subscriber=1.0.0'

In [ ]:
# artifact を S3 にアップロードする
subscriber_artifact_s3_uri=f's3://{bucket}/components/artifacts/com.example.Subscriber/1.0.0/'
!aws s3 cp ./src/ggv2/components/artifacts/com.example.Subscriber/1.0.0 {subscriber_artifact_s3_uri} --recursive
    
# recipe を S3 にアップロードする
recipe_s3_uri = f's3://{bucket}/components/recipes/'
!aws s3 cp ./src/ggv2/components/recipes/com.example.Subscriber-1.0.0.yaml {recipe_s3_uri}

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
sudo mkdir -p {base_dir}
mkdir -p {artifact_dir}
cd {artifact_dir}
aws s3 cp {subscriber_artifact_s3_uri} . --recursive
mkdir {recipe_dir}
cd {recipe_dir}
aws s3 cp {recipe_s3_uri}com.example.Subscriber-1.0.0.yaml .
sudo chown -R ubuntu:ubuntu {base_dir}
sudo /greengrass/v2/bin/greengrass-cli deployment create --recipeDir {recipe_dir} --artifactDir {artifact_base_dir} --merge "{component_name}"
"""
command_list = command_str.split('}\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

#### Subscriber のデプロイ結果を確認
* デプロイの結果確認も Publisher と同じ

In [ ]:
# deployment id を標準出力から取得する
deployment_id = response['stdout'].split(' ')[-1]
print(f'Deployment Id: {deployment_id}')

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
sudo /greengrass/v2/bin/greengrass-cli deployment status -i {deployment_id}
"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
sleep(60)
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

#### デプロイした Subscriber コンポーネントの動作を確認
推論結果は`/tmp/Greengrass_Subscriber.log`に保存するようにアプリを作っているので、中身を確認する

In [ ]:
command_str=f"""#!/bin/bash
cat /tmp/Greengrass_Subscriber.log"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

### Greengrass クラウドからステージングにデプロイ

![Greengrass クラウドからステージングにデプロイ](./image/image07.png)

* Staging を利用してデプロイのテスト
* 手順としては、クラウド側にコンポーネントを作成してデプロイ先を選択してデプロイ
* Publisher と Subscriber をそれぞれ行う

#### 既存コンポーネントが存在する場合の削除
* ハンズオンの本筋とは関係ない
* 名前の重複は許されないため事前に削除しておく

In [ ]:
result = ggv2_client.list_components()

for component in result["components"]:
    if component["componentName"] in ["com.example.Subscriber", "com.example.Publisher"]:
        response = ggv2_client.list_component_versions(
            arn=component["arn"]
        )
        for version in response["componentVersions"]:
            print(f"delete: {version['componentName']} {version['componentVersion']}")
            response = ggv2_client.delete_component(
                arn=version["arn"]
            )

#### Publisher のコンポーネントを作成
* クラウドでコンポーネントを作成する場合、recipe をファイルで渡せないため、yaml形式の文字列を作成する
* 使用する API は Greengrass v2 の [create_component_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/greengrassv2.html#GreengrassV2.Client.create_component_version) と [get_component](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/greengrassv2.html#GreengrassV2.Client.get_component)

In [ ]:
artifact_path = '{artifacts:path}'
recipe = f"""
---
RecipeFormatVersion: '2020-01-25'
ComponentName: com.example.Publisher
ComponentVersion: '1.0.0'
ComponentDescription: A component that publishes messages.
ComponentPublisher: Amazon
ComponentConfiguration:
  DefaultConfiguration:
    accessControl:
      aws.greengrass.ipc.pubsub:
        'com.example.Publisher:pubsub:1':
          policyDescription: Allows access to publish to all topics.
          operations:
            - 'aws.greengrass#PublishToTopic'
          resources:
            - '*'
Manifests:
- Name: Linux
  Platform:
    os: linux
  Lifecycle:
    Install:
        python3 -m pip install pip & pip3 install awsiotsdk numpy tensorflow-cpu==2.4.1 Pillow -U
    Run: |-
        python3 {artifact_path}/publisher.py {artifact_path}
  Artifacts:
  - Uri: "{publisher_artifact_s3_uri}publisher.py"
  - Uri: "{publisher_artifact_s3_uri}test_X.npy"
"""
publisher_component_name = yaml.safe_load(recipe)['ComponentName']
publisher_component_version = yaml.safe_load(recipe)['ComponentVersion']

In [ ]:
# コンポーネント作成
response = ggv2_client.create_component_version(
    inlineRecipe=recipe.encode(),
    tags={
        'Name': 'Publisher'
    }
)
publisher_component_vesrion_arn = response['arn']

In [ ]:
# 作成したコンポーネントを確認
response = ggv2_client.get_component(
    recipeOutputFormat='YAML',
    arn=publisher_component_vesrion_arn
)
print(response)

#### Subscriber のコンポーネントを作成
* Subscriber も Publisher 同様に yaml 形式で recipe を作成する

In [ ]:
artifact_path = '{artifacts:path}'
recipe = f"""
---
RecipeFormatVersion: "2020-01-25"
ComponentName: "com.example.Subscriber"
ComponentVersion: "1.0.0"
ComponentType: "aws.greengrass.generic"
ComponentDescription: "A component that subscribes to messages."
ComponentPublisher: "Amazon"
ComponentConfiguration:
  DefaultConfiguration:
    accessControl:
      aws.greengrass.ipc.pubsub:
        'com.example.Subscriber:pubsub:1':
          policyDescription: "Allows access to subscribe to all topics."
          operations:
          - "aws.greengrass#SubscribeToTopic"
          resources:
          - "*"
Manifests:
- Name: Linux
  Platform:
    os: linux
  Lifecycle:
    Install:
        python3 -m pip install pip & pip3 install awsiotsdk numpy tensorflow-cpu==2.4.1 Pillow -U
    Run: |-
        python3 {artifact_path}/subscriber.py {artifact_path}
  Artifacts:
  - Uri: "{subscriber_artifact_s3_uri}subscriber.py"
  - Uri: "{subscriber_artifact_s3_uri}classifier.h5"
"""
subscriber_component_name = yaml.safe_load(recipe)['ComponentName']
subscriber_component_version = yaml.safe_load(recipe)['ComponentVersion']

In [ ]:
response = ggv2_client.create_component_version(
    inlineRecipe=recipe.encode(),
    tags={
        'Name': 'subscriber'
    }
)
subscriber_component_vesrion_arn = response['arn']

In [ ]:
response = ggv2_client.get_component(
    recipeOutputFormat='YAML',
    arn=subscriber_component_vesrion_arn
)
print(response)

#### Publisher / Subscriber のクラウドデプロイ
* 事前に deploy と deploy 結果の確認用ヘルパー関数を用意しておく
* デプロイはリスト形式で複数のコンポーネントを同時に指定できる
* デプロイ先の指定は thing 単体でも thing group でも対象にできる。
* ヘルパー関数で利用する API は以下の通り
  * デプロイの作成 (Greengrass v2) : [create_deployment](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/greengrassv2.html#GreengrassV2.Client.create_deployment)
  * ジョブの確認 (IoT) : [describe_job_execution](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iot.html#IoT.Client.describe_job_execution)

In [ ]:
# deployのヘルパー
def deploy_component(target_arn, deployment_name, components):
    response = ggv2_client.create_deployment(
        targetArn=target_arn, # デプロイ先のIoT thing か group
        deploymentName=deployment_name, # デプロイの名前
        components=components,
        iotJobConfiguration={
            'timeoutConfig': {'inProgressTimeoutInMinutes': 600}
        },
        deploymentPolicies={
            'failureHandlingPolicy': 'ROLLBACK',
            'componentUpdatePolicy': {
                'timeoutInSeconds': 600,
                'action': 'NOTIFY_COMPONENTS'
            },
            'configurationValidationPolicy': {
                'timeoutInSeconds': 600
            }
        },
        tags={
            'Name': deployment_name
        }
    )
    return response

In [ ]:
# Job確認のヘルパー
def check_job_status(job_id, thing_list):
    sleep(10)
    for target in thing_list:
        retry = 20 # wait a while to the job finish
        count=0
        while True: # check for  2 minuets
            response = iot_client.describe_job_execution(
                jobId=job_id,
                thingName=target
            )
            if response["execution"]["status"] in ["QUEUED", "IN_PROGRESS"]:
                print(f'target: {target} status: {response["execution"]["status"]}')
                sleep(10)
            else:
                print(f'target: {target} status: {response["execution"]["status"]}')
                break
            if retry <= count:
                      print("time out")
                      break

##### ステージング機へのデプロイ
* デプロイ先の thin arn を取得した後、作成した deploy_component メソッド(の中で呼び出される create_deployment メソッド)でデプロイを作成する

In [ ]:
# デプロイ先の thing arn を取得
response = iot_client.describe_thing_group(
    thingGroupName=staging_group["name"]
)
staging_group_arn=response["thingGroupArn"]
print(staging_group_arn)

In [ ]:
# デプロイの作成
deployment_name = 'staging_deployment'
components={
    publisher_component_name: { # コンポーネントの名前
        'componentVersion': publisher_component_version,
    },
    subscriber_component_name: { # コンポーネントの名前
        'componentVersion': subscriber_component_version,
    },
}

response = deploy_component(staging_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], staging_group["thing_names"])

#### ステージング機へのデプロイ結果確認
* Subscriber の動作だけ確認する(Publisherが動いていないとSubscriberは何も出力しないため、Subscriber が動いていることが確認取れればPublisherも動いている）

In [ ]:
# Subscriber の確認
sleep(60)
command_str=f"""#!/bin/bash
cat /tmp/Greengrass_Subscriber.log"""
command_list = command_str.split('\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)
print('------')
command_id = send_command(bucket,instance_id_list[1],command_list)
response = check_command_result(command_id,instance_id_list[1])

##### 本番機へのデプロイ
![Greengrass クラウドから本番機にデプロイ](./image/image08.png)

* 構成はステージングの一緒のため、ステージングの図の矢印を一部省略している
* deployment の向き先だけを変える

In [ ]:
response = iot_client.describe_thing_group(
    thingGroupName=production_group["name"]
)
production_group_arn=response["thingGroupArn"]
print(production_group_arn)

In [ ]:
deployment_name = 'production_deployment'
components={
    publisher_component_name: { # コンポーネントの名前
        'componentVersion': publisher_component_version,
    },
    subscriber_component_name: { # コンポーネントの名前
        'componentVersion': subscriber_component_version,
    },
}

response = deploy_component(production_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], production_group["thing_names"])

#### 本番機へのデプロイ結果確認
* Staging と同じなので省略
* 同じコマンドを打ってみてください  

---

## コンテナを利用したコンポーネントの開発
* Greengrass で動かすアプリはコンテナでも動かすことができる
* コンテナで下記の動作をするアプリを動かす
    1. GAN で画像を生成(撮影相当)
    2. 撮影した画像をMLのモデルで異常があるか判定
    3. MQTT で判定内容をクラウドにパブリッシュ

### コンテナイメージのビルド
* ビルドする内容は`./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/`に配置済なので、Dockerfile の中身を最初に確認する
* ビルドは docker の build コマンドを利用する  

![イメージのビルド](./image/image09.png)

In [ ]:
!cat ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/Dockerfile

In [ ]:
!pygmentize ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/IoTPublisher.py

In [ ]:
# Image のビルド
image_name = 'com-example-iotpublisher'
tag = ':1'

%cd ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/
!cp ../../com.example.Publisher/1.0.0/test_X.npy ./test_X.npy
!cp ../../com.example.Subscriber/1.0.0/classifier.h5 ./classifier.h5
!docker rmi $(docker images -a -q)
!docker build -t {image_name}{tag} .
%cd ../../../../../../

### ECR へコンテナイメージをプッシュ
* 使用する ECR の API は以下
  * [get-login-password](https://docs.aws.amazon.com/cli/latest/reference/ecr/get-login-password.html)
  * [(delete-repository)](https://docs.aws.amazon.com/cli/latest/reference/ecr/delete-repository.html) ※ハンズオンの本筋とは関係ない
  * [create-repository](https://docs.aws.amazon.com/cli/latest/reference/ecr/create-repository.html)

In [ ]:
# boto3の機能を使ってリポジトリ名に必要な情報を取得する
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name
ecr_endpoint = f'{account_id}.dkr.ecr.{region}.amazonaws.com/' 
repository_uri = f'{ecr_endpoint}{image_name}'
image_uri = f'{repository_uri}{tag}'

!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {ecr_endpoint}
!docker tag {image_name}{tag} {image_uri}
# 同名のリポジトリがあった場合は削除
!aws ecr delete-repository --repository-name $image_name --force
# リポジトリを作成
!aws ecr create-repository --repository-name $image_name
# イメージをプッシュ
!docker push {image_uri}

### コンテナを使ったコンポーネントを開発機にローカルデプロイ
![コンテナをローカルデプロイ](./image/image10.png)
#### コンテナローカルデプロイ用 Recipe を作成
* Lifecycle として doccker run で起動するように指定
* Artifact として Image の URI を指定
* ローカルでデプロイするには、yaml ファイルをローカル側にファイルで配置する必要があるので、S3 を介して配置する
* 使用している API は S3 の [cp](https://docs.aws.amazon.com/cli/latest/reference/s3/cp.html)

In [ ]:
recipe = f"""---
RecipeFormatVersion: '2020-01-25'
ComponentName: com.example.IoTPublisher
ComponentVersion: '1.0.0'
ComponentDescription: Publish MQTT message to AWS IoT Core in Docker image.
ComponentPublisher: Amazon
ComponentDependencies:
  aws.greengrass.DockerApplicationManager:
    VersionRequirement: ~2.0.0
  aws.greengrass.TokenExchangeService:
    VersionRequirement: ~2.0.0
ComponentConfiguration:
  DefaultConfiguration:
    accessControl:
      aws.greengrass.ipc.mqttproxy:
        'com.example.IoTPublisher:dockerimage:latest':
          policyDescription: Allows access to publish to inference/result.
          operations:
            - 'aws.greengrass#PublishToIoTCore'
          resources:
            - 'inference/result'

Manifests:
  - Platform:
      os: all
    Lifecycle:
      Run: 'docker run --rm -v /greengrass/v2:/greengrass/v2 -e AWS_REGION=$AWS_REGION -e SVCUID=$SVCUID -e AWS_GG_NUCLEUS_DOMAIN_SOCKET_FILEPATH_FOR_COMPONENT=$AWS_GG_NUCLEUS_DOMAIN_SOCKET_FILEPATH_FOR_COMPONENT -e AWS_CONTAINER_AUTHORIZATION_TOKEN=$AWS_CONTAINER_AUTHORIZATION_TOKEN -e AWS_CONTAINER_CREDENTIALS_FULL_URI=$AWS_CONTAINER_CREDENTIALS_FULL_URI {image_uri}'
    Artifacts:
      - URI: "docker:{image_uri}"

"""
print(recipe)

In [ ]:
recipe_path = './src/ggv2/components/recipes/com.example.IotPublisher-1.0.0.yaml'
with open(recipe_path,'w') as f:
    f.write(recipe)

In [ ]:
# Publisher に必要なスクリプトとモデルを S3 にアップロード
iotpublisher_recipe_uri = f's3://{bucket}/components/recipes/com.example.IoTPublisher-1.0.0.yaml'
!aws s3 cp {recipe_path} {iotpublisher_recipe_uri}

#### 開発機でコンテナコンポーネントのローカルデプロイ
* デプロイの仕方は Publisher / Subrisher と同じ

In [ ]:
recipe_dir = f'{base_dir}/components/recipes'
component_name = 'com.example.IoTPublisher=1.0.0'

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
cd {recipe_dir}
aws s3 cp {iotpublisher_recipe_uri} ./
sudo chown -R ubuntu:ubuntu {base_dir}
sudo usermod -aG docker ggc_user
sudo /greengrass/v2/bin/greengrass-cli component stop -n com.example.Publisher
sudo /greengrass/v2/bin/greengrass-cli component stop -n com.example.Subscriber
sudo /greengrass/v2/bin/greengrass-cli deployment create --recipeDir {recipe_dir} --merge "{component_name}"
"""
command_list = command_str.split('}\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

In [ ]:
print(response)

#### ローカルデプロイした IoTPublisher が MQTT でパブリッシュした内容を確認する
* AWS マネジメントコンソールでトピックの [MQTT のテスト画面](https://ap-northeast-1.console.aws.amazon.com/iot/home?region=ap-northeast-1#/test)があるので、[./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/IoTPublisher.py](./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.0/IoTPublisher.py) にある通り、`inference/result` というトピックをサブスクライブする
* 実際にはクラウドのサービスでサブスクライブして、後段の処理を行うが、コードでサブスクライブする場合はこちらのURLを参照
  https://docs.aws.amazon.com/ja_jp/iot/latest/developerguide/sdk-tutorials.html

### コンテナを使ったコンポーネントをステージング機にクラウドデプロイ
* 先程ローカルデプロイしたレシピを流用してコンポーネントを作成する
![コンテナをステージングにクラウドデプロイ](./image/image11.png)

#### 既存コンテナコンポーネントが存在する場合の削除
* コンポーネントが重複するとエラーで落ちるので事前に削除する

In [ ]:
print(recipe)
iotpublisher_component_name = yaml.safe_load(recipe)['ComponentName']
iotpublisher_component_version = yaml.safe_load(recipe)['ComponentVersion']

In [ ]:
result = ggv2_client.list_components()

for component in result["components"]:
    if component["componentName"] in [iotpublisher_component_name]:
        response = ggv2_client.list_component_versions(
            arn=component["arn"]
        )
        for version in response["componentVersions"]:
            print(f"delete: {version['componentName']} {version['componentVersion']}")
            response = ggv2_client.delete_component(
                arn=version["arn"]
            )

#### コンテナを用いたコンポーネントの作成

In [ ]:
response = ggv2_client.create_component_version(
    inlineRecipe=recipe.encode(),
    tags={
        'Name': iotpublisher_component_name
    }
)
iotpublisher_component_vesrion_arn = response['arn']

response = ggv2_client.get_component(
    recipeOutputFormat='YAML',
    arn=iotpublisher_component_vesrion_arn
)
print(response)

#### ステージング機のデプロイの改定
* deployment はターゲット毎に決まっており、ステージング機へのデプロイは先程 Publisher / Subscriber コンポーネントをデプロイしたものがあるので、そのデプロイを IoTPublisherコンポーネントのデプロイで上書く
* デプロイに Publisher / Subscriber を明記しないことで、Publisher / Subscriber の動作も止まる

In [ ]:
# デプロイの作成
deployment_name = 'staging_deployment'
components={
    iotpublisher_component_name: { # コンポーネントの名前
        'componentVersion': iotpublisher_component_version,
    }
}

response = deploy_component(staging_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], staging_group["thing_names"])

#### ステージング機にクラウドデプロイした IoTPublisher が MQTT でパブリッシュした内容を確認する
* 先程と同様 [MQTT のテスト画面](https://ap-northeast-1.console.aws.amazon.com/iot/home?region=ap-northeast-1#/test)で、`inference/result` というトピックをサブスクライブする
* しかし、このままサブスクライブすると、困ったことに気づくので、本番機へのデプロイはやめて、コンポーネントを修正する  

---

## コンテナを利用したコンポーネントの更新
* v1.0.0 では困ったことが発生し、どのエッジデバイスがデータを送ったのかがわからない
* 区別できるようにプログラムを変更してコンポーネントを更新する
* 併せて classifier の ML モデルを高速化を目論見、SageMaker Neo でコンパイルしたモデルに差し替え、また推論コードも併せて修正する

### コンテナイメージの更新
![コンテナイメージの更新](./image/image12.png)

In [ ]:
!cat ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.1/Dockerfile

In [ ]:
!pygmentize ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.1/IoTPublisher.py

In [ ]:
# Image のビルド
image_name = 'com-example-iotpublisher'
tag = ':2' # タグを更新

%cd ./src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.1/
!cp ../../com.example.Publisher/1.0.0/test_X.npy ./test_X.npy
!docker rmi $(docker images -a -q)
!docker build -t {image_name}{tag} .
%cd ../../../../../../

### ECR へ更新したコンテナイメージをプッシュ

In [ ]:
# boto3の機能を使ってリポジトリ名に必要な情報を取得する
image_uri = f'{repository_uri}{tag}'

!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {ecr_endpoint}
!docker tag {image_name}{tag} {image_uri}
# イメージをプッシュ
!docker push {image_uri}

### 更新したコンテナを使ったコンポーネントを開発機にローカルデプロイ
#### 更新したコンテナのローカルデプロイ用 Recipe を作成
![開発機のコンテナを更新](./image/image13.png)
* バージョンを上げている
* THING_NAME を環境変数でコンテナに引き渡す
* S3 に保存しておく

In [ ]:
recipe = f"""---
RecipeFormatVersion: '2020-01-25'
ComponentName: com.example.IoTPublisher
ComponentVersion: '1.0.1'
ComponentDescription: Publish MQTT message to AWS IoT Core in Docker image.
ComponentPublisher: Amazon
ComponentDependencies:
  aws.greengrass.DockerApplicationManager:
    VersionRequirement: ~2.0.0
  aws.greengrass.TokenExchangeService:
    VersionRequirement: ~2.0.0
ComponentConfiguration:
  DefaultConfiguration:
    accessControl:
      aws.greengrass.ipc.mqttproxy:
        'com.example.IoTPublisher:dockerimage:latest':
          policyDescription: Allows access to publish all topic.
          operations:
            - 'aws.greengrass#PublishToIoTCore'
          resources:
            - 'inference/result'

Manifests:
  - Platform:
      os: all
    Lifecycle:
      Run: 'docker run --rm -v /greengrass/v2:/greengrass/v2 -e AWS_REGION=$AWS_REGION -e SVCUID=$SVCUID -e AWS_GG_NUCLEUS_DOMAIN_SOCKET_FILEPATH_FOR_COMPONENT=$AWS_GG_NUCLEUS_DOMAIN_SOCKET_FILEPATH_FOR_COMPONENT -e AWS_CONTAINER_AUTHORIZATION_TOKEN=$AWS_CONTAINER_AUTHORIZATION_TOKEN -e AWS_CONTAINER_CREDENTIALS_FULL_URI=$AWS_CONTAINER_CREDENTIALS_FULL_URI -e AWS_IOT_THING_NAME=$AWS_IOT_THING_NAME {image_uri}'
    Artifacts:
      - URI: "docker:{image_uri}"

"""
print(recipe)

In [ ]:
recipe_path = './src/ggv2/components/recipes/com.example.IotPublisher-1.0.1.yaml'
with open(recipe_path,'w') as f:
    f.write(recipe)

In [ ]:
# recipe を S3 にアップロード
iotpublisher_recipe_uri = f's3://{bucket}/components/recipes/com.example.IoTPublisher-1.0.1.yaml'
!aws s3 cp {recipe_path} {iotpublisher_recipe_uri}

#### 開発機で更新したコンテナのローカルデプロイ

In [ ]:
recipe_dir = f'{base_dir}/components/recipes'
component_name = 'com.example.IoTPublisher=1.0.1'

In [ ]:
command_str=f"""#!/bin/bash
su ubuntu
cd {recipe_dir}
aws s3 cp {iotpublisher_recipe_uri} ./
sudo chown -R ubuntu:ubuntu {base_dir}
sudo usermod -aG docker ggc_user
sudo /greengrass/v2/bin/greengrass-cli deployment create --recipeDir {recipe_dir} --merge "{component_name}"
"""
command_list = command_str.split('}\n')
print('EC2 に流すコマンド\n')
for command in command_list:
    print(command)

In [ ]:
command_id = send_command(bucket,instance_id_list[0],command_list)
response = check_command_result(command_id,instance_id_list[0])

#### 更新したコンテナのローカルデプロイ結果を確認（開発機）
* 先程と同様 [MQTT のテスト画面](https://ap-northeast-1.console.aws.amazon.com/iot/home?region=ap-northeast-1#/test)で、inference/result というトピックをサブスクライブする
* THING_NAME というキーにパブリッシュしたデバイスの THING_NAME が格納されていることを確認する
* ステージングも動いているので、THING_NAMEがないメッセージも混在している
---

### 更新したコンテナを使ったコンポーネントをステージング機にクラウドデプロイ
![ステージング機のコンテナを更新](./image/image14.png)
#### 更新したコンテナを用いたコンポーネントの作成

In [ ]:
print(recipe)
iotpublisher_component_name = yaml.safe_load(recipe)['ComponentName']
iotpublisher_component_version = yaml.safe_load(recipe)['ComponentVersion']

In [ ]:
response = ggv2_client.create_component_version(
    inlineRecipe=recipe.encode(),
    tags={
        'Name': iotpublisher_component_name
    }
)
iotpublisher_component_vesrion_arn = response['arn']

response = ggv2_client.get_component(
    recipeOutputFormat='YAML',
    arn=iotpublisher_component_vesrion_arn
)
print(response)

#### 更新したコンテナを用いたステージング機のデプロイの改定

In [ ]:
# デプロイの作成
deployment_name = 'staging_deployment'
components={
    iotpublisher_component_name: { # コンポーネントの名前
        'componentVersion': iotpublisher_component_version,
    }
}
response = deploy_component(staging_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], staging_group["thing_names"])

#### ステージング機で更新したコンテナの MQTT 動作確認
* 先程と同様 [MQTT のテスト画面](https://ap-northeast-1.console.aws.amazon.com/iot/home?region=ap-northeast-1#/test)で、inference/result というトピックをサブスクライブする
* 今度は メッセージに THING_NAME が含まれているため、どのデバイスがそのメッセージを送信したかがわかるようになっている

---

#### 更新したコンテナを用いた本番機のデプロイの改定
ステージングで動作を確認し、メッセージでどのデバイスが発したものかがわかったので、本番機にデプロイする
![本番機のコンテナを更新](./image/image15.png)

In [ ]:
# デプロイの作成
deployment_name = 'production_deployment' # 変えるのはここだけ

response = deploy_component(production_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], production_group["thing_names"])

#### 本番機で更新したコンテナの MQTT 動作確認
* 先程と同様 [MQTT のテスト画面](https://ap-northeast-1.console.aws.amazon.com/iot/home?region=ap-northeast-1#/test)で、`inference/result` というトピックをサブスクライブする
* 今度は メッセージに THING_NAME が含まれているため、どのデバイスがそのメッセージを送信したかがわかるようになっている

---

In [ ]:
# デプロイの作成
deployment_name = 'production_deployment' # 変えるのはここだけ
response = deploy_component(staging_group_arn, deployment_name, components)
check_job_status(response["iotJobId"], staging_group["thing_names"])

In [ ]:
!date